In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from spleeter.audio.adapter import get_default_audio_adapter
import librosa

audio_loader = get_default_audio_adapter()

# Runtime measurement
import time

In [3]:
# Testing with gtzan
# Samplerate 22050 Hz
# Constants
PATH = '/home/steffen/Datasets/gtzan/'
DATA_SET = PATH + 'genres_original/'
META_DATA_3_SEC = PATH + 'features_3_sec.csv'
META_DATA_30_SEC = PATH +'features_30_sec.csv'
df = pd.read_csv(META_DATA_30_SEC)
df = df[['filename', 'label']]
for (index, filename_col, label_col) in df.itertuples():
    df.loc[[index],['filename']] = DATA_SET + label_col + '/' + filename_col

df = df.rename(columns={ 'filename': 'file' })
display(df)

,file,label
0,/home/steffen/Datasets/gtzan/genres_original/b...,blues
1,/home/steffen/Datasets/gtzan/genres_original/b...,blues
2,/home/steffen/Datasets/gtzan/genres_original/b...,blues
3,/home/steffen/Datasets/gtzan/genres_original/b...,blues
4,/home/steffen/Datasets/gtzan/genres_original/b...,blues
...,...,...
995,/home/steffen/Datasets/gtzan/genres_original/r...,rock
996,/home/steffen/Datasets/gtzan/genres_original/r...,rock
997,/home/steffen/Datasets/gtzan/genres_original/r...,rock
998,/home/steffen/Datasets/gtzan/genres_original/r...,rock


In [9]:
# Testing with fma
# Samplerate "mostly" (don't know what they mean) 44100 Hz
# Constants
PATH = '/home/steffen/Datasets/fma/'
DATA_SET = os.path.join(PATH, 'fma_small/')
META_DATA = os.path.join(PATH, 'fma_metadata/tracks.csv')
df = pd.read_csv(META_DATA, index_col=0, header=[0, 1])
df = df[df[('set', 'subset')] == 'small']
df = df[[('track', 'genre_top')]]
new_df = pd.DataFrame(
    { 'file': os.path.join(DATA_SET, "{:06d}".format(index)[:3], "{:06d}".format(index) + ".mp3"), 'label': label } for (index, label) in df.itertuples()
)

df = new_df

display(df)

,file,label
0,/home/steffen/Datasets/fma/fma_small/000/00000...,Hip-Hop
1,/home/steffen/Datasets/fma/fma_small/000/00000...,Hip-Hop
2,/home/steffen/Datasets/fma/fma_small/000/00001...,Pop
3,/home/steffen/Datasets/fma/fma_small/000/00014...,Folk
4,/home/steffen/Datasets/fma/fma_small/000/00014...,Folk
...,...,...
7995,/home/steffen/Datasets/fma/fma_small/154/15430...,Hip-Hop
7996,/home/steffen/Datasets/fma/fma_small/154/15430...,Hip-Hop
7997,/home/steffen/Datasets/fma/fma_small/154/15441...,Pop
7998,/home/steffen/Datasets/fma/fma_small/154/15441...,Pop


In [95]:
print(df.iloc[0]['file'])

/home/steffen/Datasets/fma/fma_small/000/000002.mp3


In [96]:
librosa_loaded, sample_rate = librosa.load(df.iloc[0]['file'], sr=44100)

print("librosa shape & sample rate")
print(librosa_loaded.shape)
print(sample_rate)

librosa shape & sample rate
(1321967,)
44100


In [89]:
start = time.time()
spleeter_loaded = audio_loader.load(df.iloc[0]['file'], sample_rate=44100)

if spleeter_loaded[0].shape[1] == 1:
    audio = np.array([ ( 'original', np.repeat(a=spleeter_loaded[0], repeats=2, axis=1) ) ])
else: 
    audio = np.array([ ( 'original', spleeter_loaded[0]) ])

data = np.array([ df.iloc[0]['label'], np.array(audio) ])

data[1] = np.append(data[1], ['some_other', spleeter_loaded[0]])

np_data = data

end = time.time()
print(end - start)

0.06511592864990234


In [90]:
start = time.time()
spleeter_loaded = audio_loader.load(df.iloc[0]['file'], sample_rate=44100)

if spleeter_loaded[0].shape[1] == 1:
    audio = { 'original': np.repeat(a=spleeter_loaded[0], repeats=2, axis=1) }
else: 
    audio = { 'original': spleeter_loaded[0]}

data = {
    'label': df.iloc[0]['label'],
    'data': audio
}

data['data']['some_other'] = spleeter_loaded[0]

data['data'] = list(data['data'].items())
data = list(data.items())

py_data = np.array([data])

end = time.time()
print(end - start)

0.0564267635345459


In [96]:
print(np_data)
print(np_data.size*np_data.itemsize)
print()
print(py_data)
print(py_data.size*py_data.itemsize)

['Hip-Hop'
 array(['original',
       array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [-0.22564425, -0.26903775],
       [-0.18057378, -0.21723682],
       [-0.0919052 , -0.11841217]], dtype=float32),
       'some_other',
       array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [-0.22564425, -0.26903775],
       [-0.18057378, -0.21723682],
       [-0.0919052 , -0.11841217]], dtype=float32)], dtype=object)]
16

[[['label' 'Hip-Hop']
  ['data'
   list([('original', array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [-0.22564425, -0.26903775],
       [-0.18057378, -0.21723682],
       [-0.0919052 , -0.11841217]], dtype=float32)), ('some_other', array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [-0

In [3]:
class FileLoader():

    def __init__(self):
        self._audio_loader = get_default_audio_adapter()


    def load(self, df):
        collection = None
        for (_, file, label) in tqdm(df.itertuples()):
            try:
                audio = self._audio_loader.load(file, sample_rate=44100)

                if audio[0].shape[1] == 1:
                    audio = np.array([ ( 'original', np.repeat(a=audio[0], repeats=2, axis=1) ) ])
                else: 
                    audio = np.array([ ( 'original', audio[0]) ])

                data = np.array([ label, np.array(audio) ])

                if collection is None:
                    collection = np.array([data])
                else:
                    collection = np.append(collection, [data])
            except Exception as e:
                print(e)

        return collection


In [4]:
file_loader = FileLoader()
data = file_loader.load(df)

556it [00:41, 10.98it/s]An error occurs with ffprobe (see ffprobe output below)

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/steffen/miniconda3/envs/Projektarbeit --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4